In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import numpy as np
import pandas as pd
import os
from typing_extensions import TypedDict
from typing import Generator, List, NamedTuple, Iterable
import json
import cv2
import matplotlib.pyplot as plt

In [ ]:
fff = pd.read_csv('../input/data-real-frame-123/Data_real_Frame_123.csv')

l=[]
data_font=[]
real_train_size=[]

for i in range(len(fff['Image'])) :
    b=fff['Image'][i]
    b=b[1:-1].split(', ')
    b=list(map(int,b))
    b=[b]
    b=np.array(b)
    b=b.reshape(32,32,1)
    l.append(b)
    real_train_size.append(fff['size'][i])

real_train_img=np.array(l)
real_train_size=np.array(real_train_size)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import PIL
import matplotlib.pyplot
import os
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import json
def get_location(path_json,path_img):
    #output_path = '/home/superai2-4031/Hackathon1/testcrop/' # 
    images=[]
    image = cv2.imread(path_img)
    j=open(path_json)
    js=json.load(j)
    for i in range(len(js)):
        try:
            parentId = (js[i]['id'])
            x1 = round(js[i]['x'])
            x2 = round(x1+js[i]['width'])
            y1 = round(js[i]['y'])
            y2 = round(y1+js[i]['height'])
            img =np.array(image[y1:y2,x1:x2])
            w,h=img.shape[0:2]
            if w==h:  img
            elif w>h: img= img[0:h , 0:h]
            elif w<h: img= img[0:w , 0:w]
            img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img=cv2.resize(img,(32,32))
            img=img[:,:,None]
            images.append(img)
        except:
            img=np.zeros((32,32))
            img=cv2.resize(img,(32,32))
            img=img[:,:,None]
            images.append(img)
            
    return images
def get_label(path_json):
    parentId=[]
    j=open(path_json)
    js=json.load(j)
    for i in range(len(js)):
        parentId.append(js[i]['id'])
    return parentId

In [ ]:
folder=[]
parentID=[]
image=[]
path='../input/data-test2' # path test
path_final_test='../input/super-ai-engineer-2021-font-recognition/final_test' # path test
csv=pd.read_csv('../input/sample/sample_submission.csv') #path subbission
i=0
for folder in sorted(os.listdir(path)): #path test
    path_img = path +'/'+ str(folder) +'/'+ 'image.png'
    path_json= path +'/'+ str(folder) +'/'+ 'test.json'

    parentId= get_label(path_json)

    im=get_location(path_json,path_img)
    image=image+im
    parentID=parentID+parentId
    
for folder in sorted(os.listdir(path_final_test)): #path test_final
    path_img = path_final_test +'/'+ str(folder) +'/'+ 'image.png'
    path_json= path_final_test +'/'+ str(folder) +'/'+ 'test.json'
    parentId= get_label(path_json)
    im=get_location(path_json,path_img)
    image=image+im
    parentID=parentID+parentId
    
    
image=np.array(image)
parentID=np.array(parentID)

In [ ]:
size_type = ('12px', '14px', '16px', '18px', '20px', '22px', '24px')
size_type_df = pd.DataFrame(size_type, columns=['Size_type'])
size_type_df

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)

# enc_df = enc.fit_transform(family_type_df[['family_type']]).toarray()
label_size_df = enc.fit_transform(np.array(real_train_size).reshape((-1,1))).toarray()

label_size_df , len(label_size_df)

In [ ]:
index_size = enc.inverse_transform(label_size_df)
index_size , len(index_size)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(real_train_img, label_size_df, test_size=0.1, random_state=42)
X_train.shape , X_test.shape , y_train.shape , y_test.shape , X_train.shape[0]

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input ,Conv2D, Dense, Flatten, GlobalAveragePooling2D, AveragePooling2D
batch_size_ = 64
numOfLabel = 7
resnet=ResNet50(include_top=False, weights='imagenet')
x_in = layers.Input(shape=(32, 32, 1))
x = Conv2D(3, 1)(x_in)
x=resnet(x)
x=Flatten()(x)
x = layers.Dense(1000, activation='relu')(x)
#x = AveragePooling2D(pool_size=(7, 7))(x)
x = Flatten(name="flatten")(x)
x = Dense(256, activation="relu")(x)
#x = Dropout(0.5)(x)
x = layers.Dense(7, activation='softmax')(x)


model = Model(x_in, x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()
for layer in model.layers:
    layer.trainable = False
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes
steps_per_epoch_ = X_train.shape[0] // batch_size_
steps_per_epoch_

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
callback_list = [early_stopping]
csv_logger = tf.keras.callbacks.CSVLogger('training.csv', append=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.001)
history = model.fit(x=tf.squeeze(X_train), y=tf.squeeze(y_train)  ,validation_data = (X_test,y_test), steps_per_epoch = steps_per_epoch_, epochs = 100,callbacks=callback_list)
model.save("model_size.h5")

fig, axes = plt.subplots(1, 2)

# plot train and test accuracies
axes[0].plot(history.history['accuracy']) # training accuracy
axes[0].plot(history.history['val_accuracy']) # testing accuracy
axes[0].legend(['Training', 'Testing'])
axes[0].set_title('Accuracy Over Time')
axes[0].set_xlabel('epoch')
axes[0].set_ybound(0.0, 1.0)

# same plot zoomed into [0.85, 1.00]
axes[1].plot(history.history['loss']) # training accuracy
axes[1].plot(history.history['val_loss']) # testing accuracy
axes[1].legend(['Training', 'Testing'])
axes[1].set_title('Log-Inverse Accuracy')
axes[1].set_xlabel('epoch')
#axes[1].set_ybound(0.90,1.0)
plt.show()

score = model.evaluate(X_test,y_test, verbose=False)
print('Loss: {}'.format(score[0]))
print('Accuracy: {}%'.format(np.round(10000*score[1])/100))

In [ ]:
batch_size_ = 64
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras import Model 
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import gradient_descent_v2
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input ,Conv2D, Dense, Flatten, GlobalAveragePooling2D
batch_size_ = 64
numOfLabel = 7
model =load_model('../input/model-size123/model_size.h5')

In [ ]:
Y_pred = np.argmax(model.predict(image),axis=-1)
y_class=[]
for i in Y_pred:
    y_class.append(size_type[int(i)])
df=pd.DataFrame({'ID':parentID,'size':y_class})
df.to_csv('size_predicted.csv',index=False)